<a href="https://colab.research.google.com/github/Asmitha-Sa/Navigate09Project/blob/main/RAG_Legal_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu transformers sentence-transformers


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.1 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import faiss
import numpy as np


In [ ]:
legal_docs = [
    "Under California law, the statute of limitations for breach of a written contract is four years.",
    "An employer must provide reasonable accommodations to employees with disabilities under the ADA.",
    "A tenant has the right to a habitable living environment, including heat, water, and electricity.",
    "Defamation involves a false statement presented as a fact that injures a party's reputation."
]


In [ ]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(legal_docs, convert_to_tensor=False)

index = faiss.IndexFlatL2(doc_embeddings[0].shape[0])
index.add(np.array(doc_embeddings))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def retrieve_relevant_legal_docs(query, k=2):
    query_vec = embedder.encode([query])[0]
    D, I = index.search(np.array([query_vec]), k)
    return [legal_docs[i] for i in I[0]]


In [ ]:
query = "What is the statute of limitations for a contract dispute in California?"
relevant_docs = retrieve_relevant_legal_docs(query)
print("Relevant Legal Docs:\n", relevant_docs)


Relevant Legal Docs:
 ['Under California law, the statute of limitations for breach of a written contract is four years.', "Defamation involves a false statement presented as a fact that injures a party's reputation."]


In [ ]:

generator = pipeline('text-generation', model='gpt2', tokenizer='gpt2')
context = ' '.join(relevant_docs)
input_prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
response = generator(input_prompt, max_length=100, do_sample=True)[0]['generated_text']
print("\nGenerated Answer:\n", response)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Answer:
 Context: Under California law, the statute of limitations for breach of a written contract is four years. Defamation involves a false statement presented as a fact that injures a party's reputation.

Question: What is the statute of limitations for a contract dispute in California?

Answer: A contract dispute is one that requires a plaintiff to prove an unlawful misrepresentation, to prove the true facts of the claim, and to prove that his or her acts were in fact unlawful in light of his
